# Injecting Simulated Satellites to Quantify Dwarf Search Sensitivity in DP0

In [ ]:
import os
import sys
sys.path.append(os.path.expandvars('$HOME/software/simple_adl'))
import glob
import yaml

import numpy as np
import pandas as pd

import healpy as hp
import fitsio as fits

import matplotlib.pyplot as plt
import seaborn as sns

from astropy import units as u
from astropy.coordinates import SkyCoord

import simple_adl.survey
import simple_adl.isochrone
from simple_adl.coordinate_tools import distanceModulusToDistance, angsep
from simple_adl.search import write_output, search_by_distance, cut_isochrone_path

In [ ]:
#cmap = sns.cubehelix_palette(start=1.0, rot=-1.0, light=0.8, dark=0.2, hue=1.0, reverse=True, as_cmap=True)
cmap = sns.cubehelix_palette(start=3.0, rot=0.5, light=0.6, dark=0.2, hue=1.0, reverse=True, as_cmap=True)
#cmap = sns.cubehelix_palette(start=0.0, rot=0.0, light=0.7, dark=0.3, hue=1.0, reverse=True, as_cmap=True)
#cmap = sns.cubehelix_palette(start=0.5, rot=0.0, light=0.7, dark=0.3, hue=1.0, reverse=True, as_cmap=True)

cmap

In [ ]:
def gnomic(mu_ra, mu_dec, ra, dec):
    """
    https://mathworld.wolfram.com/GnomonicProjection.html
    The Gnomic projection is a conformal (angle-preserving) map of 
    coordinates on a sphere to coordinates on a plane around some central
    coordinate.
    """
    mu_ra = np.deg2rad(mu_ra)
    mu_dec = np.deg2rad(mu_dec)
    ra = np.deg2rad(ra)
    dec = np.deg2rad(dec)
    
    cos_c = np.sin(mu_dec) * np.sin(dec) + np.cos(mu_dec) * np.cos(dec) * np.sin(ra - mu_ra)
    x = np.cos(dec) * np.sin(ra - mu_ra) / cos_c
    y = (np.cos(mu_dec) * np.sin(dec) - np.sin(mu_dec) *np.cos(dec) * np.cos(ra - mu_ra)) / cos_c
    
    return x, y

## Load data from RSP TAP

In [ ]:
from lsst.rsp import get_tap_service

service = get_tap_service()
assert service is not None
# assert service.baseurl == "https://data.lsst.cloud/api/tap"
print(service.baseurl)

In [ ]:
def query(service, ra, dec, radius=1.0, gmax=23.5):
    """Return data queried from Rubin TAP
    Parameters
    ----------
    service : TAP service [str]
    ra      : Right Ascension [deg]
    dec     : Declination [deg]
    radius  : radius around (ra, dec) [deg]

    Returns
    -------
    good_results : pd.Dataframe
    """

    # Redenning coefficients
    R_g = 3.185
    R_r = 2.140
    R_i = 1.571
    
    # Define our reference position on the sky and cone radius in arcseconds
    # to use in all following examples
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    radius = radius * u.deg
    
    # Quality selection and star--galaxy separation adapted from
    # https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/object_pandas_stellar_locus.ipynb

    snr_threshold = 5
    mag_err_threshold = 1/snr_threshold
    mag_threshold = 26
    
    # bright_snr_threshold = 100
    # mag_err_threshold = 1/bright_snr_threshold
    
    safe_max_extended = 1.0
    
    query = f"""
        SELECT
            ra, dec,
            mag_g, mag_r,
            magerr_g, magerr_r,
            mag_g - {R_g} AS mag_corrected_g,
            mag_r - {R_r} AS mag_corrected_r,
            extendedness
        FROM dp01_dc2_catalogs.object
        WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', {coord.ra.value}, {coord.dec.value}, {radius.value})) = 1
        AND extendedness < {str(safe_max_extended)}
    """

    job = service.submit_job(query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    async_results = job.fetch_result()
    results = async_results.to_table().to_pandas()
    job.delete()

    good_snr = (results['magerr_g'] < mag_err_threshold) & (results['magerr_r'] < mag_err_threshold)
    good_results = results[good_snr]
    
    return good_results

## Prepare simulated satellites

In [ ]:
def get_catalog_file(catalog_dir, mc_source_id):
    """
    Inputs:
        catalog_dir = string corresponding to directory containing the stellar catalog infiles
        mc_source_id = integer corresponding the target MC_SOURCE_ID value
    Outputs:
        catalog_infile = string corresponding to filename of stellar catalog containing mc_source_id
    """
    catalog_infiles = sorted(glob.glob(catalog_dir + '/*catalog*.fits'))
    mc_source_id_array = []
    catalog_infile_index_array = []
    for ii, catalog_infile in enumerate(catalog_infiles):
        mc_source_id_min = int(os.path.basename(catalog_infile).split('.')[0].split('mc_source_id_')[-1].split('-')[0])
        mc_source_id_max = int(os.path.basename(catalog_infile).split('.')[0].split('mc_source_id_')[-1].split('-')[1])
        assert (mc_source_id_max > mc_source_id_min) & (mc_source_id_min >= 1), 'Found invalue MC_SOURCE_ID values in filenames'
        mc_source_id_array.append(np.arange(mc_source_id_min, mc_source_id_max + 1))
        catalog_infile_index_array.append(np.tile(ii, 1 + (mc_source_id_max - mc_source_id_min)))

    mc_source_id_array = np.concatenate(mc_source_id_array)
    catalog_infile_index_array = np.concatenate(catalog_infile_index_array)

    assert len(mc_source_id_array) == len(np.unique(mc_source_id_array)), 'Found non-unique MC_SOURCE_ID values in filenames'
    assert np.in1d(mc_source_id, mc_source_id_array), 'Requested MC_SOURCE_ID value not among files'
    mc_source_id_index = np.nonzero(mc_source_id == mc_source_id_array)[0][0] # second [0] added by smau 7/23/18 to fix incompatiable type bug
    return catalog_infiles[catalog_infile_index_array[mc_source_id_index]]

In [ ]:
def load_sim_data(sim_dir, mc_source_id):
    """
    Load info for injecting satellite sims
    """
    cat_file = get_catalog_file(sim_dir, mc_source_id)
    cat_fits = fits.FITS(cat_file)
    w = cat_fits[1].where(f'MC_SOURCE_ID == {mc_source_id}')
    try:       
        data = cat_fits[1][w]
        cat_fits.close()
        return data
    except IndexError: 
        print('Array is empty')
        
    return

In [ ]:
sim_dir = '/project/shared/data/satsim/lsst_dc2_v4/'
population_file = glob.glob(os.path.join(sim_dir, '*population*'))

In [ ]:
print(population_file)

In [ ]:
sim_population = fits.read(*population_file)

In [ ]:
sim_positions = np.unique(sim_population[['RA', 'DEC']])

In [ ]:
np.unique(sim_population[['RA', 'DEC']], return_counts=True)

In [ ]:
sim_population = sim_population.byteswap().newbyteorder()
pop = pd.DataFrame(sim_population)
print(pop)

### CMD plots

In [ ]:
def plot_cmd(data, axs):
    """Plot a color magnitude diagram.
    
    data: merged_data DataFrame with photometry data
    """
    y = data['mag_g']     #need to check if these are correct columns
    x = data['mag_g'] - data['mag_r']
    
    xlims = [-1, 1.5]  #need to find better way to restrict axes
    ylims = [16,28]
    axs.set_xlim(xlims); 
    axs.set_ylim(ylims); 
    
    axs.set_ylabel('$Magnitude (g)$')
    axs.set_xlabel('$Color (g-r)$')
    
    axs.plot(x, y, 'ko', markersize=0.3, alpha=0.3)
    axs.invert_yaxis()

    
    plt.show()
    
    return

## Main loop over simulated satellites per position

In [ ]:
# clear output file
f = open(os.path.join('search/results_dir','cal.csv'), 'w')
f.close()

In [ ]:
# changed functionality to only query once
fixed_position= [sim_positions[4][0], sim_positions[4][1]]
real_data = query(service, fixed_position[0], fixed_position[1], radius=2.0, gmax=23.5)  # pd.DataFrame

In [ ]:
outfile = 'out4.csv'
for position in sim_positions:
    sim_population_at_position = sim_population[sim_population[['RA', 'DEC']] == position]
    for mcid in sim_population_at_position['MC_SOURCE_ID']:
        sim_data = load_sim_data(sim_dir, mcid)
        if sim_data is not None:
            sim_data = sim_data.byteswap().newbyteorder()
            sim_data = pd.DataFrame(sim_data)   # convert to pd.DataFrame
            c2 = SkyCoord(sim_data['ra'], sim_data['dec'], unit='deg', frame='icrs')
            center = SkyCoord(fixed_position[0], fixed_position[1], unit='deg')
            d2d = center.separation(c2) 
            catalogmsk = d2d < 2*u.deg
            sim_data = sim_data[catalogmsk]
            
            if sim_data.empty:
                print("No satellites to inject into region at ({},{})".format(fixed_position[0], fixed_position[1]))
                continue
            
            frames = [real_data[real_data.columns[:-1]], sim_data[real_data.columns[:-1]]]
            merged_data = pd.concat(frames)  # pd.Dataframe
            # perform mag cut on the merged data
            good_snr = (merged_data['magerr_g'] < 0.2) & (merged_data['magerr_r'] < 0.2)
            merged_data = merged_data[good_snr]
            good_mag = (merged_data['mag_g'] < 26) & (merged_data['mag_r'] < 26)
            merged_data = merged_data[good_mag]

            fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(24, 6))

            hb = axs[0].hexbin(*gnomic(*fixed_position, real_data['ra'], real_data['dec']), mincnt=1, cmap=cmap, gridsize=50)
            cb = fig.colorbar(hb, ax=axs[0])
            # axs[0].scatter(0, 0, c='r', marker='+', s=200)
            axs[0].set_title('DC2 Data')
            axs[0].set_xlabel(r'$x$')
            axs[0].set_ylabel(r'$y$')

            hb = axs[1].hexbin(*gnomic(*fixed_position, sim_data['ra'], sim_data['dec']), mincnt=1, cmap=cmap, gridsize=50)
            cb = fig.colorbar(hb, ax=axs[1])
            # axs[1].scatter(0, 0, c='r', marker='+', s=200)
            axs[1].set_title('Simulated Satellite')
            axs[1].set_xlabel(r'$x$')
            axs[1].set_ylabel(r'$y$')

            hb = axs[2].hexbin(*gnomic(*fixed_position, merged_data['ra'], merged_data['dec']), mincnt=1, cmap=cmap, gridsize=50)
            cb = fig.colorbar(hb, ax=axs[2])
            # axs[2].scatter(0, 0, c='r', marker='+', s=200)
            axs[2].set_title('Merged')
            axs[2].set_xlabel(r'$x$')
            axs[2].set_ylabel(r'$y$')

            plot_cmd(merged_data, axs[3])
            plt.show()

            print('Searching for: MC_SOURCE_ID ', mcid)
            with open('search/config.yaml') as ymlfile:
                cfg = yaml.load(ymlfile, Loader=yaml.SafeLoader)
                survey = simple_adl.survey.Survey(cfg)
            ra = fixed_position[0]
            dec = fixed_position[1]
            region = simple_adl.survey.Region(survey, ra, dec)
            region.data = merged_data

            # Scan in distance moduli    
            distance_modulus_search_array = np.arange(16., survey.catalog['mag_max'], 0.5)

            iso_search_array = [simple_adl.isochrone.Isochrone(survey=survey.isochrone['survey'],
                                                   band_1=survey.band_1.lower(),
                                                   band_2=survey.band_2.lower(),
                                                   age=12.0, #survey.isochrone['age'],
                                                   metallicity=0.00010, #survey.isochrone['metallicity'],
                                                   distance_modulus=distance_modulus)
                                for distance_modulus in distance_modulus_search_array]

            iso_selection_array = [cut_isochrone_path(region.data[survey.mag_dered_1], 
                                                  region.data[survey.mag_dered_2],
                                                  region.data[survey.mag_err_1],
                                                  region.data[survey.mag_err_2],
                                                  iso,
                                                  survey.catalog['mag_max'],
                                                  radius=0.1)
                               for iso in iso_search_array]

            results = [search_by_distance(survey, region, distance_modulus, iso_sel) for (distance_modulus,iso_sel) in zip(distance_modulus_search_array,iso_selection_array)] 
            best_ra_peak, best_dec_peak, best_r_peak, best_distance_modulus, n_obs_peak, n_obs_half_peak, n_model_peak, best_sig_peak = 0, 0, 0, 0, 0, 0, 0, 0
            for scan in results:
                ra_peak_array, dec_peak_array, r_peak_array, sig_peak_array, distance_modulus_array, n_obs_peak_array, n_obs_half_peak_array, n_model_peak_array = np.asarray(scan)
                if mcid: 
                    mc_source_id_array = np.full_like(distance_modulus_array, mcid)
                else:
                    mc_source_id_array = np.zeros(len(distance_modulus_array))

                # Sort peaks according to significance
                index_sort = np.argsort(sig_peak_array)[::-1]
                ra_peak_array = ra_peak_array[index_sort]
                dec_peak_array = dec_peak_array[index_sort]
                r_peak_array = r_peak_array[index_sort]
                sig_peak_array = sig_peak_array[index_sort]
                distance_modulus_array = distance_modulus_array[index_sort]
                n_obs_peak_array = n_obs_peak_array[index_sort]
                n_obs_half_peak_array = n_obs_half_peak_array[index_sort]
                n_model_peak_array = n_model_peak_array[index_sort]
                mc_source_id_array = mc_source_id_array[index_sort]



                # Collect overlapping peaks
                for ii in range(0, len(sig_peak_array)):
                    if sig_peak_array[ii] < 0:
                        continue
                    sep = angsep(ra_peak_array[ii], dec_peak_array[ii], ra_peak_array, dec_peak_array)
                    sig_peak_array[(sep < r_peak_array[ii]) & (np.arange(len(sig_peak_array)) > ii)] = -1.
                    #sig_peak_array[(sep < 0.5) & (np.arange(len(sig_peak_array)) > ii)] = -1. # 0.5 deg radius

                # Prune the list of peaks
                ra_peak_array = ra_peak_array[sig_peak_array > 0.]
                dec_peak_array = dec_peak_array[sig_peak_array > 0.]
                r_peak_array = r_peak_array[sig_peak_array > 0.]
                distance_modulus_array = distance_modulus_array[sig_peak_array > 0.]
                n_obs_peak_array = n_obs_peak_array[sig_peak_array > 0.]
                n_obs_half_peak_array = n_obs_half_peak_array[sig_peak_array > 0.]
                n_model_peak_array = n_model_peak_array[sig_peak_array > 0.]
                mc_source_id_array = mc_source_id_array[sig_peak_array > 0.]
                sig_peak_array = sig_peak_array[sig_peak_array > 0.] # Update the sig_peak_array last!

                if sig_peak_array[0] > best_sig_peak:
                    best_sig_peak = sig_peak_array[0]
                    best_ra_peak = ra_peak_array[0]
                    best_dec_peak = dec_peak_array[0]
                    best_r_peak = r_peak_array[0]
                    best_distance_modulus = distance_modulus_array[0]
                    n_obs_peak = n_obs_peak_array[0]
                    n_obs_half_peak = n_obs_half_peak_array[0]
                    n_model_peak = n_model_peak_array[0]
                    mc_source_id = mc_source_id_array[0]

                for ii in range(0, len(sig_peak_array)):
                    print('{:0.2f} sigma; (RA, Dec) = ({:0.2f}, {:0.2f}); r = {:0.2f} deg; d = {:0.1f}, mu = {:0.2f} mag, mc_source_id: {:0.2f}'.format(sig_peak_array[ii], 
                             ra_peak_array[ii], 
                             dec_peak_array[ii], 
                             r_peak_array[ii],
                             distanceModulusToDistance(distance_modulus_array[ii]),
                             distance_modulus_array[ii],
                             mc_source_id_array[ii]))
            del merged_data
            # Write output
            # there is an error that comes every now and agin that changes the output format
        try:
            if (len(sig_peak_array) > 0):
                write_output(os.path.join('search', survey.output['results_dir']), survey.catalog['nside'], region.pix_center, best_ra_peak, best_dec_peak,
                                best_r_peak, best_distance_modulus, 
                                n_obs_peak, n_obs_half_peak, n_model_peak, 
                                best_sig_peak, mcid, 0, outfile)
            else:
                print('No significant hotspots found.')
                nan_array = [np.nan]
                write_output(os.path.join('search', survey.output['results_dir']), survey.catalog['nside'], region.pix_center,
                                 nan_array, nan_array, nan_array, nan_array, 
                                 nan_array, nan_array, nan_array, nan_array,
                                 [mc_source_id], 0, outfile)
        except Exception as e: 
            print(e)
            print('Data missing, cannot write to file')

In [ ]:
# calibration
# need to consider how we store this information
outfile = 'cal.csv'

good_snr = (real_data['magerr_g'] < 0.2) & (real_data['magerr_r'] < 0.2)
the_data = real_data[good_snr]
good_mag = (the_data['mag_g'] < 26) & (the_data['mag_r'] < 26)
the_data = the_data[good_mag]

with open('search/config.yaml') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.SafeLoader)
    survey = simple_adl.survey.Survey(cfg)
    
ra = fixed_position[0]
dec = fixed_position[1]
region = simple_adl.survey.Region(survey, ra, dec)
region.data = the_data

# Scan in distance moduli    
distance_modulus_search_array = np.arange(16., survey.catalog['mag_max'], 0.5)

iso_search_array = [simple_adl.isochrone.Isochrone(survey=survey.isochrone['survey'],
                                       band_1=survey.band_1.lower(),
                                       band_2=survey.band_2.lower(),
                                       age=12.0, #survey.isochrone['age'],
                                       metallicity=0.00010, #survey.isochrone['metallicity'],
                                       distance_modulus=distance_modulus)
                    for distance_modulus in distance_modulus_search_array]

iso_selection_array = [cut_isochrone_path(region.data[survey.mag_dered_1], 
                                      region.data[survey.mag_dered_2],
                                      region.data[survey.mag_err_1],
                                      region.data[survey.mag_err_2],
                                      iso,
                                      survey.catalog['mag_max'],
                                      radius=0.1)
                   for iso in iso_search_array]

results = [search_by_distance(survey, region, distance_modulus, iso_sel) for (distance_modulus,iso_sel) in zip(distance_modulus_search_array,iso_selection_array)] 
best_ra_peak, best_dec_peak, best_r_peak, best_distance_modulus, n_obs_peak, n_obs_half_peak, n_model_peak, best_sig_peak = 0, 0, 0, 0, 0, 0, 0, 0

for scan in results:
    ra_peak_array, dec_peak_array, r_peak_array, sig_peak_array, distance_modulus_array, n_obs_peak_array, n_obs_half_peak_array, n_model_peak_array = np.asarray(scan)
    # Sort peaks according to significance
    index_sort = np.argsort(sig_peak_array)[::-1]
    ra_peak_array = ra_peak_array[index_sort]
    dec_peak_array = dec_peak_array[index_sort]
    r_peak_array = r_peak_array[index_sort]
    sig_peak_array = sig_peak_array[index_sort]
    distance_modulus_array = distance_modulus_array[index_sort]
    n_obs_peak_array = n_obs_peak_array[index_sort]
    n_obs_half_peak_array = n_obs_half_peak_array[index_sort]
    n_model_peak_array = n_model_peak_array[index_sort]

    # Collect overlapping peaks
    for ii in range(0, len(sig_peak_array)):
        if sig_peak_array[ii] < 0:
            continue
        sep = angsep(ra_peak_array[ii], dec_peak_array[ii], ra_peak_array, dec_peak_array)
        sig_peak_array[(sep < r_peak_array[ii]) & (np.arange(len(sig_peak_array)) > ii)] = -1.
        #sig_peak_array[(sep < 0.5) & (np.arange(len(sig_peak_array)) > ii)] = -1. # 0.5 deg radius

    # Prune the list of peaks
    ra_peak_array = ra_peak_array[sig_peak_array > 0.]
    dec_peak_array = dec_peak_array[sig_peak_array > 0.]
    r_peak_array = r_peak_array[sig_peak_array > 0.]
    distance_modulus_array = distance_modulus_array[sig_peak_array > 0.]
    n_obs_peak_array = n_obs_peak_array[sig_peak_array > 0.]
    n_obs_half_peak_array = n_obs_half_peak_array[sig_peak_array > 0.]
    n_model_peak_array = n_model_peak_array[sig_peak_array > 0.]
    sig_peak_array = sig_peak_array[sig_peak_array > 0.] # Update the sig_peak_array last!

    if sig_peak_array[0] > best_sig_peak:
        best_sig_peak = sig_peak_array[0]
        best_ra_peak = ra_peak_array[0]
        best_dec_peak = dec_peak_array[0]
        best_r_peak = r_peak_array[0]
        best_distance_modulus = distance_modulus_array[0]
        n_obs_peak = n_obs_peak_array[0]
        n_obs_half_peak = n_obs_half_peak_array[0]
        n_model_peak = n_model_peak_array[0]
        
    for ii in range(0, len(sig_peak_array)):
        print('{:0.2f} sigma; (RA, Dec) = ({:0.2f}, {:0.2f}); r = {:0.2f} deg; d = {:0.1f}, mu = {:0.2f} mag'.format(sig_peak_array[ii], 
                 ra_peak_array[ii], 
                 dec_peak_array[ii], 
                 r_peak_array[ii],
                 distanceModulusToDistance(distance_modulus_array[ii]),
                 distance_modulus_array[ii]))

# Write output

try:
    if (len(sig_peak_array) > 0):
        write_output(os.path.join('search', survey.output['results_dir']), survey.catalog['nside'], region.pix_center, best_ra_peak, best_dec_peak,
                        best_r_peak, best_distance_modulus, 
                        n_obs_peak, n_obs_half_peak, n_model_peak, 
                        best_sig_peak, 0, 0, outfile)
    else:
        print('No significant hotspots found.')
        nan_array = [np.nan]
        write_output(os.path.join('search', survey.output['results_dir']), survey.catalog['nside'], region.pix_center,
                         nan_array, nan_array, nan_array, nan_array, 
                         nan_array, nan_array, nan_array, nan_array,
                         [0], 0, outfile)
except Exception as e: 
    print(e)
    print('Data missing, cannot write to file')

In [ ]:
# reproduce the fig 9 rom the paper

In [ ]:
# original
i = 0
for position in sim_positions:
    if i > 10:
        break
    else:
        sim_population_at_position = sim_population[sim_population[['RA', 'DEC']] == position]
        real_data = query(service, position[0], position[1], radius=2.0, gmax=23.5)  # pd.DataFrame
        sim_data = load_sim_data(sim_dir, sim_population_at_position['MC_SOURCE_ID'].item()) # np.ndarray, .item() only retrieves first record 
        if sim_data is not None:
            # to filter sim_data to only those within real_data region
            sim_data = sim_data.byteswap().newbyteorder()            # to deal with data that were created on a machine with a different byte order than the one on which you are running Python
            sim_data = pd.DataFrame(sim_data)   # convert to pd.DataFrame
            c2 = SkyCoord(sim_data['ra'], sim_data['dec'], unit='deg', frame='icrs')
            center = SkyCoord(position[0], position[1], unit='deg')
            d2d = center.separation(c2) 
            catalogmsk = d2d < 2*u.deg
            sim_data = sim_data[catalogmsk]

            frames = [real_data[real_data.columns[:-1]], sim_data[real_data.columns[:-1]]]
            merged_data = pd.concat(frames)  # pd.Dataframe
            good_snr = (merged_data['magerr_g'] < 0.2) & (merged_data['magerr_r'] < 0.2)          # perform mag cut on the merged data
            merged_data = merged_data[good_snr]
            good_mag = (merged_data['mag_g'] < 26) & (merged_data['mag_r'] < 26)
            merged_data = merged_data[good_mag]

            fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(24, 6))

            hb = axs[0].hexbin(*gnomic(*position, real_data['ra'], real_data['dec']), mincnt=1, cmap=cmap, gridsize=50)
            cb = fig.colorbar(hb, ax=axs[0])
            # axs[0].scatter(0, 0, c='r', marker='+', s=200)
            axs[0].set_title('DC2 Data')
            axs[0].set_xlabel(r'$x$')
            axs[0].set_ylabel(r'$y$')

            hb = axs[1].hexbin(*gnomic(*position, sim_data['ra'], sim_data['dec']), mincnt=1, cmap=cmap, gridsize=50)
            cb = fig.colorbar(hb, ax=axs[1])
            # axs[1].scatter(0, 0, c='r', marker='+', s=200)
            axs[1].set_title('Simulated Satellite')
            axs[1].set_xlabel(r'$x$')
            axs[1].set_ylabel(r'$y$')

            hb = axs[2].hexbin(*gnomic(*position, merged_data['ra'], merged_data['dec']), mincnt=1, cmap=cmap, gridsize=50)
            cb = fig.colorbar(hb, ax=axs[2])
            # axs[2].scatter(0, 0, c='r', marker='+', s=200)
            axs[2].set_title('Merged')
            axs[2].set_xlabel(r'$x$')
            axs[2].set_ylabel(r'$y$')

            plot_cmd(merged_data, axs[3])
            plt.show()
    i +=1
        #break #for testing purposes, we don't want to run the full loop
    